Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
import joblib
import matplotlib.pylab as pl
import seaborn as sns
import shap
from scipy.stats import pearsonr

In [2]:
rand_seed = 0

Reading the data

In [3]:
train_data = pd.read_csv("./Data/prepped_train.csv")
test_data = pd.read_csv("./Data/prepped_test.csv")
target = pd.read_csv("./Data/prepped_target.csv")
pred_base = pd.read_csv("./Data/prediction_base.csv")

target.set_index("Id", inplace = True)

Label encoding the features

In [4]:
enc = LabelEncoder()
train_result = {}
test_result = {}
for col in train_data.columns:
    if (train_data[col].dtype == "int" or train_data[col].dtype == "float"):
        train_result[col] = train_data[col]
        test_result[col] = test_data[col]
    else:
        train_result[col] = pd.Series(enc.fit_transform(train_data[col]))
        test_result[col] = pd.Series(enc.transform(test_data[col]))
train_features = pd.DataFrame(train_result)
test_features = pd.DataFrame(test_result)

train_features.set_index("Id", inplace = True)
test_features.set_index("Id", inplace = True)

XGBoost with early stopping

In [5]:
np.random.seed(rand_seed)
train_features_grid, train_features_validation, target_grid, target_validation = train_test_split(train_features, target, test_size = 0.1)

In [6]:
xgb = XGBRegressor(n_jobs = -1, booster = "gbtree", n_estimators = 1000, colsample_bytree = 0.5)
eta_range = np.linspace(0.01, 0.5, 50, endpoint = True)
max_depth_range = range(1, 11)
xgb_params = {'max_depth': max_depth_range, 'eta': eta_range}
gs_xgb = GridSearchCV(xgb, xgb_params, cv = 10, return_train_score = True)

In [7]:
np.random.seed(rand_seed)
fit_params={"early_stopping_rounds" : 20, 
            "eval_metric" : "mae", 
            "eval_set" : [[train_features_validation, target_validation]]}
%time gs_xgb.fit(train_features_grid, target_grid,verbose = 0, **fit_params)

CPU times: user 1h 18min 46s, sys: 7min 20s, total: 1h 26min 7s
Wall time: 11min 4s


GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster='gbtree',
                                    colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=0.5,
                                    gamma=None, gpu_id=None,
                                    importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=1000, n_...
                                    tree_method=None, validate_parameters=None,
                                    verbosity=None),
             param_grid={'eta': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19,

In [8]:
gs_xgb.best_score_

0.5953724913027496

Second round, narrowing in on eta

In [9]:
max_d = gs_xgb.best_params_["max_depth"]
xgb = XGBRegressor(n_jobs = -1, booster = "gbtree", n_estimators = 1000, colsample_bytree = 0.5, max_depth = max_d)
eta_mid = gs_xgb.best_params_["eta"]
eta_range = np.linspace(eta_mid - 0.01, eta_mid + 0.01, 200, endpoint = False)
xgb_params = {'eta': eta_range}
gs_xgb = GridSearchCV(xgb, xgb_params, cv = 10, return_train_score = True)

In [10]:
np.random.seed(rand_seed)
fit_params={"early_stopping_rounds" : 20, 
            "eval_metric" : "mae", 
            "eval_set" : [[train_features_validation, target_validation]]}
%time gs_xgb.fit(train_features_grid, target_grid,verbose = 0, **fit_params)

CPU times: user 21min 21s, sys: 1min 57s, total: 23min 18s
Wall time: 2min 59s


GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster='gbtree',
                                    colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=0.5,
                                    gamma=None, gpu_id=None,
                                    importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=3, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=1000, n_job...
       0.1952, 0.1953, 0.1954, 0.1955, 0.1956, 0.1957, 0.1958, 0.1959,
       0.196 , 0.1961, 0.1962, 0.1963, 0.1964, 0.1965, 0.1966, 0.1967,
       0.1968, 0.1969, 0.197 , 0.1971, 0.1972, 0.1973, 0.1974, 0.1975,
       0.1976, 0.1977, 0.1978, 0.1979, 0.198 , 0.1981, 0.1982, 0.1983,
       0

In [11]:
gs_xgb.best_score_

0.6019180520362657

XGBoost outcomes

In [12]:
print("max depth:", max_d)
print("eta:", gs_xgb.best_params_["eta"])

max depth: 3
eta: 0.19590000000000002


In [13]:
importances = pd.Series(gs_xgb.best_estimator_.feature_importances_, index=train_features.columns).sort_values(ascending=False)
importances[:20]

Neighborhood          0.092270
CentralAir            0.079328
BasementQualFactor    0.060826
MSSubClass            0.053235
SaleCondition         0.050202
PoolQC                0.045812
MSZoning              0.041520
TotLivArea            0.041160
GarageQual            0.038825
OverallCond           0.038557
OverallQual           0.036321
HeatingQC             0.032654
HalfBath              0.031550
LotArea               0.031317
ExterCond             0.028876
Fireplaces            0.028838
GarageCars            0.025918
Foundation            0.025452
AgeRemod              0.023431
LotShape              0.016953
dtype: float32

Running the model on the test dataset, taking exponential as predicted values are log prices

In [14]:
model_xgb = gs_xgb.best_estimator_

In [15]:
pred_xgb = pd.Series(model_xgb.predict(test_features))
pred_combined = np.exp(np.sum([pred_xgb, pred_base["predictions"]],axis=0))

In [16]:
prediction = pd.DataFrame({"Id" : test_data["Id"], "SalePrice" : pred_combined})
prediction.set_index("Id", inplace = True)

In [17]:
prediction.to_csv("./Data/prediction.csv")

Save encoder and model for later

In [ ]:
filename = 'fitted_model.sav'
joblib.dump(model_xgb, filename)
filename = 'fitted_encoder.sav'
joblib.dump(enc, filename)

In [ ]:
train_features.to_csv("./Data/encoded_train_data.csv")

In [ ]:
shap_values = shap.TreeExplainer(model_xgb).shap_values(train_features)

In [ ]:
shap.summary_plot(shap_values, train_features)

In [ ]:
shap.dependence_plot("TotLivArea", shap_values, train_features)

In [ ]:
shap.dependence_plot("OverallQual", shap_values, train_features)

In [ ]:
shap.dependence_plot("LotArea", shap_values, train_features)

In [ ]:
shap.dependence_plot("Neighborhood", shap_values, train_features)

In [ ]:
shap_interaction_values = shap.TreeExplainer(model_xgb).shap_interaction_values(train_features)

In [ ]:
shap.summary_plot(shap_interaction_values, train_features)

In [ ]:
shap.dependence_plot(
    ("TotLivArea", "TotLivArea"),
    shap_interaction_values, train_features,
    display_features = train_features
)

In [ ]:
shap.dependence_plot(
    ("TotLivArea", "OverallQual"),
    shap_interaction_values, train_features,
    display_features = train_features
)

In [ ]:
shap.dependence_plot(
    ("Neighborhood", "OverallQual"),
    shap_interaction_values, train_features,
    display_features = train_features
)

In [ ]:
tmp = np.abs(shap_interaction_values).sum(0)
for i in range(tmp.shape[0]):
    tmp[i,i] = 0
inds = np.argsort(-tmp.sum(0))[:50]
tmp2 = tmp[inds,:][:,inds]
pl.figure(figsize=(12,12))
pl.imshow(tmp2)
pl.yticks(range(tmp2.shape[0]), train_features.columns[inds], rotation=50.4, horizontalalignment="right")
pl.xticks(range(tmp2.shape[0]), train_features.columns[inds], rotation=50.4, horizontalalignment="left")
pl.gca().xaxis.tick_top()
pl.show()

In [ ]:
def heatmap(valuesDF, threshold):
    # Make a mask to only show the lower left part of the table
    mask = np.triu(np.ones_like(valuesDF, dtype = bool))

    cmap = sns.diverging_palette(220, 20, as_cmap = True)

    # Draw the heatmap with the mask and correct aspect ratio
    visual = sns.heatmap(valuesDF, mask = mask, cmap = cmap, center = threshold,
                square = True, linewidths = .5, annot = True);

    return visual